In [ ]:
from importlib import reload
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats

# Add the project root to the path
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, project_root)

import pacing_auction.data as data
import pacing_auction.auction as auction
import pacing_auction.elimination as elimination
import pacing_auction.generator as generator
reload(data)
reload(elimination)
reload(auction)
reload(generator)

sns.set_theme(style="whitegrid")

plt.rcParams.update({
    'font.size': 12,           # Default font size
    'axes.titlesize': 16,      # Title font size
    'axes.labelsize': 14,      # Axis label font size
    'xtick.labelsize': 12,     # X-axis tick label size
    'ytick.labelsize': 12,     # Y-axis tick label size
    'legend.fontsize': 12,     # Legend font size
})

plt.rcParams['figure.dpi'] = 200

OUTPUT_PATH = "/Users/khalid/Desktop/honours-project-writeup/figures/"


In [ ]:
def sample(delta, sigma, mu=0.5) -> float:
    comp_mu = mu - delta if np.random.rand() < 0.5 else mu + delta
    a, b = (0 - comp_mu) / sigma, (1 - comp_mu) / sigma
    return stats.truncnorm.rvs(a, b, loc=comp_mu, scale=sigma) # type: ignore

# Parameters
delta_values = [0, 0.25, 0.5]
sigma = 0.1
n_samples = 1000
samples = [list[float]() for _ in delta_values]


In [ ]:
# Plot histograms
for i, delta in enumerate(delta_values):
    if len(samples[i]) < n_samples:
        samples[i] = [sample(delta, sigma) for _ in range(n_samples)]


fig, axes = plt.subplots(1, len(delta_values), figsize=(15, 5), sharey=True)
for i, (ax, delta) in enumerate(zip(axes, delta_values)):
    sns.histplot(samples[i], bins=50, kde=True, ax=ax)
    ax.set_title(f"δ = {delta:.2f}")
    ax.set_xlim(0, 1)

axes[0].set_ylabel("Density")
plt.tight_layout()
# plt.savefig(OUTPUT_PATH + "mixed_gaussian_samples.png")
plt.show()

In [ ]:
n, m, q = 5, 5, 1000
sim = auction.Auction(n, m, q, rng=np.random.default_rng(0))
bids = sim.bids()
bidder = 1
utils = []
for alpha in range(q + 1):
    x, p = sim.auction(adjustment=(bidder, alpha))
    utils.append(sim.utility(x, p)[bidder])

plt.figure(figsize=(10, 5))
plt.title(f"Utility of bidder 1 for different values of alpha_q")
plt.xlabel("alpha_q")
plt.ylabel("utility")
sns.lineplot(x=range(len(utils)), y=utils)
plt.tight_layout()
# plt.savefig(OUTPUT_PATH + "utility-alpha-q.png")


In [ ]:
plt.figure(figsize=(16, 10))
a = auction.Auction(10, 10, 1000, rng=np.random.default_rng(0))
res = a.responses()
print(res)
utility = res.stats["utility"]
plt.title("Utility of Bidders After Every Best-Response")
plt.xlabel("Iteration")
plt.ylabel("Utility")

for i in range(n):
    utility_i = utility[i]
    sns.lineplot(x=range(len(utility_i)), y=utility_i, label=f"Bidder {i + 1}")

plt.legend()
plt.tight_layout()
# plt.savefig(OUTPUT_PATH + "utility-best-response.png")
plt.show()

In [ ]:
def load_data(path: str):
    df = pd.read_csv(path + "/data.csv")
    df_matrices = np.load(path + "/auction_matrices.npz", allow_pickle=True)
    print(f"Matrices: {df_matrices.files}")

    return df

In [ ]:
df1 = load_data("../results/v3.1")

In [ ]:
df2 = load_data("../results/v3.2")

In [ ]:
df = pd.concat([df1, df2])
# df = pd.read_csv("../results/2025-04-03-15:25:04/data.csv")

In [ ]:
df['time_per_iteration'] = df['runtime'] / df['iterations']
df

In [ ]:
print(f"Timeouts: {df[df["timeout"]].shape[0]}")
print(f"Errors: {df[df["error"]].shape[0]}")

In [ ]:
df_time_per_iteration_pivot = df.pivot_table(
    index='n', columns='m', values='time_per_iteration', aggfunc='median'
)

plt.figure(figsize=(10, 8))
sns.heatmap(df_time_per_iteration_pivot, annot=True, cmap="YlGnBu", cbar_kws={'label': 'Time per Iteration (s)'})
plt.title('Heatmap of Time per Iteration by n and m')
plt.xlabel('m')
plt.ylabel('n')
plt.savefig(OUTPUT_PATH + "time_per_iteration_heatmap.png")
plt.show()

In [ ]:
complete = df[df["generator"] == "complete"]
sampled = df[df["generator"] == "sampled"]
correlated = df[df["generator"] == "correlated"]

elim_all = df[df["elim_strategy"] == "all"]
elim_subsequent = df[df["elim_strategy"] == "subsequent"]
elim_current = df[df["elim_strategy"] == "current"]

print(complete.shape)
print(sampled.shape)
print(correlated.shape)

print(elim_all.shape)
print(elim_subsequent.shape)
print(elim_current.shape)

In [ ]:
from matplotlib.colors import Normalize


complete_pivot = complete.pivot_table(
    index='n', columns='m', values='result_type', aggfunc=lambda x: (x == 'PNE').mean()
)
sampled_pivot = sampled.pivot_table(
    index='n', columns='m', values='result_type', aggfunc=lambda x: (x == 'PNE').mean()
)
correlated_pivot = correlated.pivot_table(
    index='n', columns='m', values='result_type', aggfunc=lambda x: (x == 'PNE').mean()
)

# Create a figure with a custom layout for three heatmaps and one colorbar
fig = plt.figure(figsize=(24, 8))

# Calculate the width ratios to include space for the colorbar
grid_spec = fig.add_gridspec(1, 4, width_ratios=[1, 1, 1, 0.05])

# Create three subplots for the heatmaps
ax1 = fig.add_subplot(grid_spec[0, 0])
ax2 = fig.add_subplot(grid_spec[0, 1])
ax3 = fig.add_subplot(grid_spec[0, 2])
cbar_ax = fig.add_subplot(grid_spec[0, 3])  # Subplot for the shared colorbar

# Find the global min and max values across all three datasets for consistent color scale
vmin = min(complete_pivot.min().min(), sampled_pivot.min().min(), correlated_pivot.min().min())
vmax = max(complete_pivot.max().max(), sampled_pivot.max().max(), correlated_pivot.max().max())
norm = Normalize(vmin=vmin, vmax=vmax)

# Plot the first heatmap (Complete) without its own colorbar
sns.heatmap(complete_pivot, annot=True, cmap="YlGnBu",
            cbar=False, ax=ax1, norm=norm)
ax1.set_title('Percentage of PNE (Complete)')
ax1.set_xlabel('m')
ax1.set_ylabel('n')

# Plot the second heatmap (Sampled) without its own colorbar
sns.heatmap(sampled_pivot, annot=True, cmap="YlGnBu",
            cbar=False, ax=ax2, norm=norm)
ax2.set_title('Percentage of PNE (Sampled)')
ax2.set_xlabel('m')
ax2.set_ylabel('n')

# Plot the third heatmap (Correlated) without its own colorbar
sns.heatmap(correlated_pivot, annot=True, cmap="YlGnBu",
            cbar=False, ax=ax3, norm=norm)
ax3.set_title('Percentage of PNE (Correlated)')
ax3.set_xlabel('m')
ax3.set_ylabel('n')

# Create a colorbar in the dedicated axis
cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap="YlGnBu"),
                   cax=cbar_ax)
cbar.set_label('Percentage of PNE')

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the figure with all three heatmaps and shared colorbar

plt.savefig(OUTPUT_PATH + "heatmap_pne.png")

plt.show()

In [ ]:
# Create pivot tables for the number of iterations
complete_iterations_pivot = complete.pivot_table(
    index='n', columns='m', values='iterations', aggfunc='median'
)
sampled_iterations_pivot = sampled.pivot_table(
    index='n', columns='m', values='iterations', aggfunc='median'
)
correlated_iterations_pivot = correlated.pivot_table(
    index='n', columns='m', values='iterations', aggfunc='median'
)

# Create a figure with a custom layout for three heatmaps and one colorbar
fig = plt.figure(figsize=(24, 8))

# Calculate the width ratios to include space for the colorbar
grid_spec = fig.add_gridspec(1, 4, width_ratios=[1, 1, 1, 0.05])

# Create three subplots for the heatmaps
ax1 = fig.add_subplot(grid_spec[0, 0])
ax2 = fig.add_subplot(grid_spec[0, 1])
ax3 = fig.add_subplot(grid_spec[0, 2])
cbar_ax = fig.add_subplot(grid_spec[0, 3])  # Subplot for the shared colorbar

# Find the global min and max values across all three datasets for consistent color scale
vmin = min(complete_iterations_pivot.min().min(), sampled_iterations_pivot.min().min(), correlated_iterations_pivot.min().min())
vmax = max(complete_iterations_pivot.max().max(), sampled_iterations_pivot.max().max(), correlated_iterations_pivot.max().max())
norm = plt.Normalize(vmin=vmin, vmax=vmax)

# Plot the first heatmap (Complete) without its own colorbar
sns.heatmap(complete_iterations_pivot, annot=True, cmap="YlGnBu",
            cbar=False, ax=ax1, norm=norm)
ax1.set_title('Median Iterations (Complete)')
ax1.set_xlabel('m')
ax1.set_ylabel('n')

# Plot the second heatmap (Sampled) without its own colorbar
sns.heatmap(sampled_iterations_pivot, annot=True, cmap="YlGnBu",
            cbar=False, ax=ax2, norm=norm)
ax2.set_title('Median Iterations (Sampled)')
ax2.set_xlabel('m')
ax2.set_ylabel('n')

# Plot the third heatmap (Correlated) without its own colorbar
sns.heatmap(correlated_iterations_pivot, annot=True, cmap="YlGnBu",
            cbar=False, ax=ax3, norm=norm)
ax3.set_title('Median Iterations (Correlated)')
ax3.set_xlabel('m')
ax3.set_ylabel('n')

# Create a colorbar in the dedicated axis
cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap="YlGnBu"),
                   cax=cbar_ax)
cbar.set_label('Median Iterations')

# Adjust layout to prevent overlap
plt.tight_layout()

plt.savefig(OUTPUT_PATH + "heatmap_iterations.png")

# Show the figure with all three heatmaps and shared colorbar
plt.show()

In [ ]:
pnes = df[df['result_type'] == 'PNE']
cycles = df[df['result_type'] == 'Cycle']

display(pnes["iterations"].describe())
display(cycles["iterations"].describe())
print(f"PNEs: {pnes.shape[0]}, {round(pnes.shape[0] * 100 / df.shape[0], 2)}%")

plt.figure(figsize=(16, 5))

# First subplot for PNEs
plt.subplot(1, 2, 1)
sns.histplot(pnes[pnes['iterations'] < 1000]['iterations'], bins=100, kde=True) # type: ignore
plt.xlim(0, 1000)
plt.title('Iteration Count Distribution for PNEs')
plt.xlabel('Iterations')
plt.ylabel('Frequency')

# Second subplot for Cycles
plt.subplot(1, 2, 2)
sns.histplot(cycles[cycles["iterations"] < 1000]['iterations'], bins=100, kde=True) # type: ignore
plt.xlim(0, 1000)
plt.title('Iteration Count Distribution for Cycles')
plt.xlabel('Iterations')
plt.ylabel('Frequency')

# Get the maximum y-limit from both subplots
ax1 = plt.gcf().axes[0]
ax2 = plt.gcf().axes[1]

plt.tight_layout()
plt.savefig(OUTPUT_PATH + "iteration_count_pne.png")
plt.show()

In [ ]:
low_pnes = pnes[pnes['iterations'] < 10]
display(low_pnes["n"].value_counts())
display(low_pnes["m"].value_counts())

In [ ]:

# Filter out iterations above 1000 to keep the plot readable
filtered_df = df[df['iterations'] < 1000]
plt.figure(figsize=(16, 10))

# Plot histograms for different values of n (keeping m constant)
plt.subplot(1, 2, 1)
for n_val in [2, 4, 6, 8, 10]:
    subset = filtered_df[filtered_df['n'] == n_val]
    if not subset.empty:
        sns.kdeplot(subset['iterations'], label=f'n={n_val}', fill=True, alpha=0.2) # type: ignore
plt.xlim(0, 1000)
plt.title('Iteration Count Distribution by n Values')
plt.xlabel('Iterations')
plt.ylabel('Frequency')
plt.legend()

# Plot histograms for different values of m (keeping n constant)
plt.subplot(1, 2, 2)
for m_val in [2, 4, 6, 8, 10]:
    subset = filtered_df[filtered_df['m'] == m_val]
    if not subset.empty:
        sns.kdeplot(subset['iterations'], label=f'm={m_val}', fill=True, alpha=0.2) # type: ignore
plt.xlim(0, 1000)
plt.title('Iteration Count Distribution by m Values')
plt.xlabel('Iterations')
plt.ylabel('Frequency')
plt.legend()

# Get the maximum y-limit from both subplots
ax1 = plt.gcf().axes[0]
ax2 = plt.gcf().axes[1]
y_max = max(ax1.get_ylim()[1], ax2.get_ylim()[1])

# Set the same y-limit for both subplots
ax1.set_ylim(0, y_max)
ax2.set_ylim(0, y_max)

plt.tight_layout()
# plt.savefig(OUTPUT_PATH + "iteration_count_by_n_m.png")
plt.show()

In [ ]:
from matplotlib.ticker import FuncFormatter
# Your existing code up until the plotting part...

plt.figure(figsize=(16, 10))

# Calculate percentage of PNEs for each sigma-delta combination
sigma_delta_pne = []
for (sigma, delta), df_group in correlated.groupby(["sigma", "delta"]):
    pnes_group = df_group[df_group['result_type'] == 'PNE']
    pne_percentage = (pnes_group.shape[0] / df_group.shape[0]) * 100

    sigma_delta_pne.append({'sigma': sigma, 'delta': delta, 'pne_percentage': pne_percentage})

# Convert to DataFrame
df_sigma_delta = pd.DataFrame(sigma_delta_pne)


# Get unique sigma values
sigma_values = sorted(df_sigma_delta['sigma'].unique())

# Plot a line for each sigma value
for sigma in sigma_values:
    sigma_data = df_sigma_delta[df_sigma_delta['sigma'] == sigma]
    sigma_data = sigma_data.sort_values('delta')
    plt.plot(sigma_data['delta'], sigma_data['pne_percentage'], marker='o', linewidth=2, label=f'σ = {sigma}')

# Format y-axis to show percentage
def percentage_formatter(x, p):
    return f'{x:.0f}%'

plt.gca().yaxis.set_major_formatter(FuncFormatter(percentage_formatter))

plt.title('Percentage of PNEs by Delta (δ) for Each Sigma (σ) Value', fontsize=16)
plt.xlabel('Delta (δ)', fontsize=14)
plt.ylabel('Percentage of PNEs (%)', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(title='Sigma Values', fontsize=12)
plt.tight_layout()
# plt.savefig(OUTPUT_PATH + "pne_percentage_by_sigma_delta.png")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.ticker import FuncFormatter

# Create a figure with subplots (2x2 grid for different delta values)
fig, axs = plt.subplots(2, 2, figsize=(16, 12))
axs = axs.flatten()

# Get unique delta values and sort them
delta_values = sorted(correlated['delta'].unique())

# Define a consistent color palette
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

# Create histograms for each delta value (PNEs only)
for i, delta in enumerate(delta_values):
    # Filter data for this delta and only for PNEs, with iterations ≤ 1000
    delta_data = correlated[(correlated['delta'] == delta) &
                           (correlated['result_type'] == 'PNE') &
                           (correlated['sigma'] == 0.0) &
                           (correlated['iterations'] <= 1000)]

    # Plot histogram of iterations
    sns.histplot(
        delta_data['iterations'],
        bins=30,
        kde=True,
        ax=axs[i],
        color=colors[i],
        alpha=0.7,
        stat='count'
    )

    # Calculate and display statistics
    median_iter = delta_data['iterations'].median()
    mean_iter = delta_data['iterations'].mean()

    # Add vertical lines for mean and median
    axs[i].axvline(median_iter, color='red', linestyle='--',
                  label=f'Median: {median_iter:.1f}')
    axs[i].axvline(mean_iter, color='black', linestyle='-',
                  label=f'Mean: {mean_iter:.1f}')

    # Calculate PNE percentage out of total for this delta
    total_delta = len(correlated[correlated['delta'] == delta])
    pne_percentage = (len(delta_data) / total_delta) * 100

    # Set title and styling
    axs[i].set_title(f'δ = {delta}: PNE Iterations Distribution ({pne_percentage:.1f}% convergence)',
                    fontsize=14)
    axs[i].set_xlabel('Iterations', fontsize=12)
    axs[i].set_ylabel('Count', fontsize=12)
    axs[i].grid(True, linestyle='--', alpha=0.7)
    axs[i].legend()

    # Optional: add text annotation with detailed statistics
    stats_text = (f"n={len(delta_data)}\n"
                 f"Min: {delta_data['iterations'].min()}\n"
                 f"Max: {delta_data['iterations'].max()}\n"
                 f"PNE rate: {pne_percentage:.1f}%")
    axs[i].text(0.95, 0.95, stats_text, transform=axs[i].transAxes,
               fontsize=10, va='top', ha='right',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

# Add a super title
plt.suptitle('Distribution of Iteration Counts for PNEs by Delta (δ) Parameter\n(Iterations ≤ 1000)',
            fontsize=16, y=0.98)

# Adjust layout
plt.tight_layout()
plt.subplots_adjust(top=0.93)

# Save or show
# plt.savefig(OUTPUT_PATH + "pne_iteration_distribution_by_delta.png", dpi=300)
plt.show()